In [1]:
# !pip install optuna
# !pip install xgboost

In [2]:
import optuna
import xgboost as xg
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as iplot
import cufflinks as cf
cf.go_offline()

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
### loading the Admission Prediction dataset

df = pd.read_csv("Admission_Predict.csv")
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [4]:
df.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [5]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,200.500000,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,115.614301,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,100.750000,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,200.500000,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,300.250000,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,400.000000,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


In [6]:
### Null val check

df.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [7]:
# features

X = df.drop(columns=["Serial No.", "Chance of Admit "])
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
395,324,110,3,3.5,3.5,9.04,1
396,325,107,3,3.0,3.5,9.11,1
397,330,116,4,5.0,4.5,9.45,1
398,312,103,3,3.5,4.0,8.78,0


In [8]:
# Label

Y = df["Chance of Admit "]

In [9]:
### Standard Scaling the data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled

array([[ 1.76210664,  1.74697064,  0.79882862, ...,  1.16732114,
         1.76481828,  0.90911166],
       [ 0.62765641, -0.06763531,  0.79882862, ...,  1.16732114,
         0.45515126,  0.90911166],
       [-0.07046681, -0.56252785, -0.07660001, ...,  0.05293342,
        -1.00563118,  0.90911166],
       ...,
       [ 1.15124883,  1.41704229,  0.79882862, ...,  1.16732114,
         1.42900622,  0.90911166],
       [-0.41952842, -0.72749202, -0.07660001, ...,  0.61012728,
         0.30403584, -1.09997489],
       [ 1.41304503,  1.58200646,  0.79882862, ...,  0.61012728,
         1.78160888,  0.90911166]])

In [10]:
## For cross validation

def objective(trial, data=X_scaled, target=Y):
    
    try:
        # splitting into test and train subsets
        x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=.1, random_state=42)

        params = {

#             # for GPU support
#             'tree_method': 'gpu_hist',

            ## "conservative" algorithms are ones that more tightly match the expected results.
            # They have less fuzziness, so they return fewer results. 
            # L2 regularization term on weights. Increasing this value will make model more conservative.
            'lambda': trial.suggest_float('lambda', 1e-4, 10),

            # L1 regularization term on weights. Increasing this value will make model more conservative.
            'alpha': trial.suggest_float('alpha', 1e-4, 10),

            ## colsample_bytree is the subsample ratio of columns when constructing each tree. 
            # Subsampling occurs once for every tree constructed.
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', np.arange(.1, 1.1, 0.1)),

            # Subsample ratio of the training instances across all the training models.
            'subsample': trial.suggest_categorical('subsample', np.arange(.1, 1.1, 0.1)),

            'learning_rate': trial.suggest_categorical('learning_rate', [0.00001, .0003, .008, .002, .01, 1, 10, 20  ]),
            'n_estimators': 3000, # number of trees in RF
            'max_depth': trial.suggest_categorical('max_depth', range(3, 13)),
            'random_state': trial.suggest_categorical('random_state', [10, 20, 30, 42, 100]),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 200)
        }

        # building a model
        mod = xg.XGBRFRegressor(**params)
        
        # by the end of each trial, that particualr config will be evaluated on the `eval_test`.
        mod.fit(x_train, y_train, eval_set=[(x_test, y_test)], verbose=True)
        pred = mod.predict(x_test)
        rmse = mean_squared_error(y_test, pred)

        return rmse

    except Exception as e:
        print(e)
        
# Best Params (after trials):
{'lambda': 7.98725011679705,
 'alpha': 0.040527141040718306,
 'colsample_bytree': 0.7000000000000001,
 'subsample': 0.6,
 'learning_rate': 1,
 'max_depth': 6,
 'random_state': 30,
 'min_child_weight': 2}

{'lambda': 7.98725011679705,
 'alpha': 0.040527141040718306,
 'colsample_bytree': 0.7000000000000001,
 'subsample': 0.6,
 'learning_rate': 1,
 'max_depth': 6,
 'random_state': 30,
 'min_child_weight': 2}

- ### colsample_bytree: 
By limiting the number of features for building each tree we may end up with trees that gained different insights from the data. They learn how to optimise for the target variable using different set of features.

- ### learning_rate:
The learning rate is the shrinkage you do at every step you are making. If you make 1 step at eta = 1.00, the step weight is 1.00. If you make 1 step at eta = 0.25, the step weight is 0.25.

- ### subsample
 **Subsample ratio of the training instances.** Say, setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees and this will prevent overfitting. Subsampling will occur once in every boosting iteration.

In [11]:
find_params = optuna.create_study(direction='minimize')
find_params.optimize(objective, n_trials=10)

best_params = find_params.best_trial.params
best_params

[I 2022-10-30 21:10:55,310] A new study created in memory with name: no-name-a4b02981-b6de-4826-ab9a-619fbfb7414d


[0]	validation_0-rmse:0.25218


[I 2022-10-30 21:10:55,766] Trial 0 finished with value: 0.063595 and parameters: {'lambda': 7.441947143075148, 'alpha': 0.599864406054624, 'colsample_bytree': 0.8, 'subsample': 0.2, 'learning_rate': 1, 'max_depth': 4, 'random_state': 42, 'min_child_weight': 154}. Best is trial 0 with value: 0.063595.


[0]	validation_0-rmse:4.03689


[I 2022-10-30 21:10:56,289] Trial 1 finished with value: 16.296506213184312 and parameters: {'lambda': 8.269218486596788, 'alpha': 5.15706132160563, 'colsample_bytree': 1.0, 'subsample': 0.7000000000000001, 'learning_rate': 20, 'max_depth': 4, 'random_state': 42, 'min_child_weight': 29}. Best is trial 0 with value: 0.063595.


...

[0]	validation_0-rmse:0.25218


[I 2022-10-30 21:10:59,097] Trial 8 finished with value: 0.063595 and parameters: {'lambda': 7.4404689463287905, 'alpha': 6.676925344575097, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.0003, 'max_depth': 5, 'random_state': 10, 'min_child_weight': 63}. Best is trial 3 with value: 0.06281013877773241.


[0]	validation_0-rmse:1.52127


[I 2022-10-30 21:10:59,532] Trial 9 finished with value: 2.3142484501317018 and parameters: {'lambda': 2.5609411347809123, 'alpha': 1.8240060503540394, 'colsample_bytree': 0.6, 'subsample': 0.1, 'learning_rate': 10, 'max_depth': 11, 'random_state': 100, 'min_child_weight': 3}. Best is trial 3 with value: 0.06281013877773241.


{'lambda': 2.2267533315391055,
 'alpha': 1.8750909816892043,
 'colsample_bytree': 0.8,
 'subsample': 1.0,
 'learning_rate': 0.008,
 'max_depth': 11,
 'random_state': 100,
 'min_child_weight': 127}

In [12]:
### Trials description

find_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.063595,2022-10-30 21:10:55.312236,2022-10-30 21:10:55.765333,0 days 00:00:00.453097,0.599864,0.8,7.441947,1.0000,4,154,42,0.2,COMPLETE
1,1,16.296506,2022-10-30 21:10:55.768325,2022-10-30 21:10:56.288528,0 days 00:00:00.520203,5.157061,1.0,8.269218,20.0000,4,29,42,0.7,COMPLETE
2,2,0.062925,2022-10-30 21:10:56.290530,2022-10-30 21:10:56.750231,0 days 00:00:00.459701,4.319490,0.6,3.498535,0.0080,6,100,10,0.6,COMPLETE
3,3,0.062810,2022-10-30 21:10:56.753231,2022-10-30 21:10:57.151473,0 days 00:00:00.398242,1.875091,0.8,2.226753,0.0080,11,127,100,1.0,COMPLETE
4,4,0.657634,2022-10-30 21:10:57.153474,2022-10-30 21:10:57.539502,0 days 00:00:00.386028,9.278231,0.8,4.543609,10.0000,6,46,42,0.2,COMPLETE
5,5,0.063595,2022-10-30 21:10:57.541503,2022-10-30 21:10:57.909530,0 days 00:00:00.368027,3.301352,0.6,9.437616,0.0003,8,195,100,0.3,COMPLETE
6,6,0.063595,2022-10-30 21:10:57.911531,2022-10-30 21:10:58.284561,0 days 00:00:00.373030,6.249818,0.2,6.148885,0.0080,9,138,20,0.2,COMPLETE
7,7,0.062877,2022-10-30 21:10:58.287560,2022-10-30 21:10:58.660858,0 days 00:00:00.373298,7.908387,0.4,6.188201,0.0100,3,144,42,0.6,COMPLETE
8,8,0.063595,2022-10-30 21:10:58.662858,2022-10-30 21:10:59.095954,0 days 00:00:00.433096,6.676925,0.5,7.440469,0.0003,5,63,10,0.8,COMPLETE
9,9,2.314248,2022-10-30 21:10:59.099954,2022-10-30 21:10:59.531985,0 days 00:00:00.432031,1.824006,0.6,2.560941,10.0000,11,3,100,0.1,COMPLETE


In [13]:
## Optimization plot

optuna.visualization.plot_optimization_history(find_params)

In [14]:
### Optimization plot (params wise)

optuna.visualization.plot_slice(find_params)

In [15]:
# best params

best_params

{'lambda': 2.2267533315391055,
 'alpha': 1.8750909816892043,
 'colsample_bytree': 0.8,
 'subsample': 1.0,
 'learning_rate': 0.008,
 'max_depth': 11,
 'random_state': 100,
 'min_child_weight': 127}

In [16]:
## Contour plot

optuna.visualization.plot_contour(find_params, params=["alpha", "lambda"])

### # Building `XGBoost Regeressor` model with the best params computed via `Optuna` study:

In [17]:
## Splitting the data into train, test sets

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=42)

In [18]:
mod = xg.XGBRFRegressor(**best_params)
mod.fit(x_train, y_train)

XGBRFRegressor(alpha=1.8750909816892043, base_score=0.5, booster='gbtree',
               callbacks=None, colsample_bylevel=1, colsample_bytree=0.8,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               lambda=2.2267533315391055, learning_rate=0.008, max_bin=256,
               max_cat_to_onehot=4, max_delta_step=0, max_depth=11,
               max_leaves=0, min_child_weight=127, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=0,
               num_parallel_tree=100, objective='reg:squarederror',
               predictor='auto', random_state=100, ...)

In [19]:
## Training accuracy

mod.score(x_train, y_train)

-2.6374203186032505

In [20]:
## Test set accuracy

mod.score(x_test, y_test)

-1.8495779608733067

### NOTE: `Optuna` supporrts all libraries out there and that's why it's li'l more popular.

## # `XGBosst Classifier` using best params computed by the `Optuna` study:

In [21]:
ls

 Volume in drive D is Local Disk 
 Volume Serial Number is 407D-3EE5

 Directory of D:\data-science\machine-learning\in10__boosting

30-10-2022  21:10    <DIR>          .
30-10-2022  21:10    <DIR>          ..
30-10-2022  20:41    <DIR>          .ipynb_checkpoints
28-09-2019  00:27            12,903 Admission_Predict.csv
28-09-2019  00:27            16,176 Admission_Predict_Ver1.1.csv
30-10-2022  21:10           359,481 in1__XGBoost, Optuna (hyperparameter tuning).ipynb
03-11-2021  01:26           390,514 wine-quality-white-and-red.csv
               4 File(s)        779,074 bytes
               3 Dir(s)  87,533,260,800 bytes free


In [22]:
## Loading the wine-qualoty dataset

df1 = pd.read_csv("wine-quality-white-and-red.csv")
df1

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [23]:
### separating out the `Wine Quality --Red` data subset

df = df1[df1['type'] == 'red'].reset_index(drop=True).drop("type", axis=1)
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [24]:
## description

df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [25]:
### Check for null values

df.isna().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

No null vals whatsoever!

In [77]:
## Feature Atrributes

x = df.drop("quality", axis=1)

X = np.array(x)
X.shape

(1599, 11)

In [62]:
## Label attribute

y = df["quality"]
y

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64

### Gotta to do `Encoding` for Label attribute:

In [63]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y)

LabelEncoder()

In [76]:
## Encoded Label attribute

Y_encoded = encoder.transform(y)

Y = Y_encoded.reshape(-1, 1)
Y.shape

(1599, 1)

In [78]:
### Train-Test split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=42)

In [102]:
### Hyperparameter tuning using `Optuna`
## Function for getting best params via trials

def objective_classification(trial):
    params = {
        'verbosity': 2,
        'objective': 'binary:logistics',
        'booster': trial.suggest_categorical('booster', ['dart', 'gbtree', 'gblinear']),
        'lambda': trial.suggest_float('lambda', 1e-4, 1),
        'alpha': trial.suggest_float('alpha', 1e-4, 1),
        'subsample': trial.suggest_float('subsample', .1, .5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', .1, .5)
    }
    
    ### There will be different params for a specific booster too!
    ## Then, let's not try to confuse the optuna.
    if params['booster'] in ['gbtree', 'dart']:
        params['gamma'] : trial.suggest_float('gamma', 1e-3, 4)
        params['eta'] : trial.suggest_float('eta', .001, 5) # learning rate
        
    mod1 = xg.XGBRFClassifier(**params)
    mod1.fit(x_train, y_train, eval_set=[(x_test, y_test)], verbose=True)
    pred = mod1.predict(x_test)
    accuracy = mod1.score(x_test, y_test)
        
    return accuracy

- ### DART booster
XGBoost mostly combines a huge number of regression trees with a small learning rate. In this situation, trees added early are significant and trees added late are unimportant.

In [103]:
## Creating a study for trials

find_params = optuna.create_study(direction='minimize')
find_params.optimize(objective_classification, n_trials=100)

best_params = find_params.best_trial.params
best_params

[I 2022-10-30 21:59:27,014] A new study created in memory with name: no-name-eea33ee9-9058-49a3-9ff7-521c27557d15


[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned

[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=5
[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6
[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=6
[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=6
[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[21:59:27] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 

[I 2022-10-30 21:59:27,661] Trial 0 finished with value: 0.534375 and parameters: {'booster': 'dart', 'lambda': 0.08536259621101763, 'alpha': 0.6465464227202442, 'subsample': 0.48142899577332965, 'colsample_bytree': 0.15108973039440055}. Best is trial 0 with value: 0.534375.


[21:59:27] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bynode", "colsample_bytree", "num_parallel_tree", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mlogloss:1.32053


...

[21:59:42] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bynode", "colsample_bytree", "num_parallel_tree", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mlogloss:1.31616


[I 2022-10-30 21:59:42,837] Trial 98 finished with value: 0.409375 and parameters: {'booster': 'gblinear', 'lambda': 0.9067271135292171, 'alpha': 0.1625863276720425, 'subsample': 0.15912991665987808, 'colsample_bytree': 0.40404296868046624}. Best is trial 1 with value: 0.40625.


[21:59:42] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bynode", "colsample_bytree", "num_parallel_tree", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mlogloss:1.31540


[I 2022-10-30 21:59:42,898] Trial 99 finished with value: 0.425 and parameters: {'booster': 'gblinear', 'lambda': 0.40127255772190124, 'alpha': 0.10748403365144787, 'subsample': 0.3004444805659889, 'colsample_bytree': 0.35336627424956296}. Best is trial 1 with value: 0.40625.


{'booster': 'gblinear',
 'lambda': 0.19355152820703483,
 'alpha': 0.6558380570156551,
 'subsample': 0.2961099170842192,
 'colsample_bytree': 0.20398965918112652}

### for `10 trials`:

In [97]:
## best_params 

"""
{'booster': 'gblinear',
 'lambda': 0.8253859153587969,
 'alpha': 0.03963805273217429,
 'subsample': 0.2826168971239176,
 'colsample_bytree': 0.34604849552914263}
"""

"\n{'booster': 'gblinear',\n 'lambda': 0.8253859153587969,\n 'alpha': 0.03963805273217429,\n 'subsample': 0.2826168971239176,\n 'colsample_bytree': 0.34604849552914263}\n"

### for `100 trials`:

In [104]:
## best_params 

best_params

{'booster': 'gblinear',
 'lambda': 0.19355152820703483,
 'alpha': 0.6558380570156551,
 'subsample': 0.2961099170842192,
 'colsample_bytree': 0.20398965918112652}

### # Building `XGBoost Classfier` model with the `best_params`:

In [105]:
mod1 = xg.XGBRFClassifier(**best_params)
mod1.fit(x_train, y_train)

[22:01:56] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bynode", "colsample_bytree", "num_parallel_tree", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRFClassifier(alpha=0.6558380570156551, base_score=0.5, booster='gblinear',
                callbacks=None, colsample_bylevel=None,
                colsample_bytree=0.20398965918112652,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, gamma=None, gpu_id=-1, grow_policy=None,
                importance_type=None, interaction_constraints=None,
                lambda=0.19355152820703483, max_bin=None,
                max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
                max_leaves=None, min_child_weight=None, missing=nan,
                monotone_constraints=None, n_estimators=100, n_jobs=0,
                num_parallel_tree=None, objective='multi:softprob',
                predictor=None, random_state=0, reg_alpha=0, ...)

In [107]:
## training accuracy:

mod1.score(x_train, y_train)

0.43080531665363564

In [108]:
## test accuracy:

mod1.score(x_test, y_test)

0.40625